In [1]:
import pandas as pd
from tqdm.auto import tqdm
from pycparser import parse_file
from pycparser.plyparser import ParseError
import numpy as np
import os
import clang.cindex
import subprocess

In [3]:
codes = pd.read_csv('../Datasets Code Snippets/post_with_only_code_snippets_final.csv')

In [6]:
codes

,id,code
0,37637877,void closeBuySell(int buyTicket){\n double b...
1,57596800,"#include <stdio.h>\n\nvoid sum(int *, int *, i..."
2,22667586,"#include ""buraco/game/board/model/KingChatFilt..."
3,22667586,"#include ""buraco\game/board/model\CardGroup.h""..."
4,55014685,#include <stdlib.h> /* EXIT malloc free */\n#i...
...,...,...
452666,64735134,"template <class T, class ...Ts>\nvoid printA(T..."
452667,65106538,#include <iostream>\nusing namespace std;\n\nv...
452668,25117355,#include <iostream>\nusing namespace std;\n\ni...
452669,55782153,#include<stdio.h>\n#include <limits.h>\n#inclu...


# Filter codes by keywords

In [5]:
words_to_search = ['strcpy', 'strncpy', 'strcat', 'strncat', 'system', 'memcpy', 'memset', 'malloc', 'gets', 'vfork', 
                   'realloc', 'pthread_mutex_lock', 'free', 'chroot', 'strlen', 'vsprintf', 'sprintf', 'scanf', 'fscanf', 
                   'sscanf', 'vscanf', 'vsscanf', 'vfscanf','snprintf', 'atoi', 'strtok', 'strcmp', 'strncmp', 'strcasecmp',
                    'strncasecmp', 'memcmp', 'signal', 'va_arg', 'vulnerab', 'fault', 'defect', 'sanit', 'mistake', 'flaw', 
                   'bug', 'infinite', 'loop', 'secur', 'overflow', 'error', 'remote', 'mitigat', 'realloc', 'heap', 
                   'privilege', 'underflow', 'attack', 'DoS', 'denial-of-service', 'initiali', 'xss', 'leak', 'patch',
                    'authori', 'corruption', 'crash', 'memory', 'null', 'injection', 'out-of-bounds', 'use-after-free', 
                   'dereferenc', 'buffer', 'hack', 'segment', 'authentication', 'exploit', 'incorrect', 'vulnerab', 'harm', 
                   'undefine', 'unpredict', 'unsafe', 'secur', 'malicious', 'dangerous', 'critical', 'bad', 'unprivileged', 
                   'negative', 'stable', 'invalid', 'flaw', 'hack', 'fix', 'change', 'modify', 'exploit', 'mitigat', 'leak', 
                   'realloc', 'invoke', 'inject', 'ensure', 'reject', 'initiali', 'fail', 'authori', 'update', 'attack', 
                   'trigger', 'lock', 'corrupt', 'crash', 'prevent', 'avoid', 'access', 'cause', 'overflow', 'terminat'              
                  ]

In [14]:
dict_keywords = dict()
idx = 0

for word in tqdm(words_to_search):
    dict_word= dict()
    for index in range(codes.shape[0]):
        if word in codes['code'][index]:
            dict_word[idx]={"id": codes['id'][index], "code": codes['code'][index]}
            idx=idx+1
    
    print(len(dict_word), " codes contain word ",word)
    dict_keywords = {**dict_keywords, **dict_word}

9869  codes contain word  strcpy
1907  codes contain word  strncpy
2749  codes contain word  strcat
281  codes contain word  strncat
13871  codes contain word  system
5841  codes contain word  memcpy
6992  codes contain word  memset
30558  codes contain word  malloc
14012  codes contain word  gets
43  codes contain word  vfork
3766  codes contain word  realloc
1249  codes contain word  pthread_mutex_lock
22123  codes contain word  free
7  codes contain word  chroot
16676  codes contain word  strlen
117  codes contain word  vsprintf
4792  codes contain word  sprintf
32076  codes contain word  scanf
4363  codes contain word  fscanf
2475  codes contain word  sscanf
1  codes contain word  vscanf
7  codes contain word  vsscanf
15  codes contain word  vfscanf
1767  codes contain word  snprintf
5058  codes contain word  atoi
3465  codes contain word  strtok
7427  codes contain word  strcmp
818  codes contain word  strncmp
184  codes contain word  strcasecmp
31  codes contain word  strncasecmp

In [15]:
len(dict_keywords)

481800

In [41]:
df_dict_keywords = pd.DataFrame.from_dict(dict_keywords, orient='index')

In [42]:
df_dict_keywords = df_dict_keywords.drop_duplicates().reset_index(drop=True)

In [43]:
df_dict_keywords

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,40233070,/***** PLEASE DO NOT EVER DO THIS IN REAL CO...
2,29652839,#include <Windows.h>\n#include <string>\n\nboo...
3,55118246,#include <cstdlib>\n#include <cstring>\n#inclu...
4,53767640,void dir_details(int socket) {\n //Set up a...
...,...,...
219569,29433869,void MainWindow::Quit()\n{\n close();\n ...
219570,49517511,#include <iostream>\n#include <string>\n#inclu...
219571,22337006,#include <iostream>\n#include<stdio.h>\n#inclu...
219572,48448538,"#include ""stdafx.h""\n#include < iostream>\n#in..."


In [44]:
df_dict_keywords.to_csv('../Datasets Code Snippets/code_snippets_static_analysis.csv', index=False)

# Detect syntax errors

In [2]:
df_dict_keywords = pd.read_csv('../Datasets Code Snippets/code_snippets_static_analysis.csv')

In [49]:
df_dict_keywords

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,40233070,/***** PLEASE DO NOT EVER DO THIS IN REAL CO...
2,29652839,#include <Windows.h>\n#include <string>\n\nboo...
3,55118246,#include <cstdlib>\n#include <cstring>\n#inclu...
4,53767640,void dir_details(int socket) {\n //Set up a...
...,...,...
219569,29433869,void MainWindow::Quit()\n{\n close();\n ...
219570,49517511,#include <iostream>\n#include <string>\n#inclu...
219571,22337006,#include <iostream>\n#include<stdio.h>\n#inclu...
219572,48448538,"#include ""stdafx.h""\n#include < iostream>\n#in..."


In [108]:
dictionary = df_dict_keywords.to_dict(orient='index')

In [54]:
for index in tqdm(range(len(dictionary))):
    file_name = f"{index}.cpp"
    with open("tmp/"+file_name, "w", encoding="utf-8") as f:
        f.write(dictionary[index]['code'])

In [2]:
proc = subprocess.Popen(["echo", "%PATH%"], stdout=subprocess.PIPE, shell=True)
(out, err) = proc.communicate()
result = out.decode('utf-8').split(";")
path_to_clang = ""

for env in result:
    if 'Llvm' in env:
        path_to_clang = env
        

In [4]:
tmp_no_header = set(os.listdir('tmp_no_header'))
file_to_analyze = [x for x in os.listdir('tmp') if x not in tmp_no_header]

for file in tqdm(file_to_analyze):
    if not os.path.exists("tmp_no_header/"+file):
        try:
            proc = subprocess.Popen([path_to_clang+"/clang++", "-E", "tmp/"+file , "-o", "output.txt"], stdout=subprocess.PIPE, shell=True, bufsize=1)
            (out, err) = proc.communicate()

            lines_to_write = []

            if os.path.exists("output.txt"):
                for line in reversed(list(open("output.txt"))):
                    if not line.startswith('#'):
                        lines_to_write.append(line)
                    else:
                        break;
                
                with open("tmp_no_header/"+file, "w", encoding="utf-8") as f:
                    for line in reversed(lines_to_write):
                        f.write(line)
                os.remove("output.txt")
            else:
                os.remove("tmp/"+file) #remove the file in case of error with clang
                
        except UnicodeDecodeError:
            print(file)
            break;

In [46]:
files = os.listdir('tmp_no_header')
index_to_delete=[]

for file in tqdm(files):
    try:
        parse_file(os.path.join('tmp_no_header', file))
    except ParseError:
        index_to_delete.append(file.split('.')[0])
    except AssertionError:
        print("AssertionError")
        print(file)
        break;
    except UnicodeDecodeError:
        print("UnicodeDecodeError")
        print(file)
        break;
    except TypeError:
        print("TypeError")
        print(file)
        break;
    except AttributeError:
        print("AttributeError")
        print(file)
        break;

In [109]:
len(index_to_delete)

126934

In [110]:
len(dictionary)

219574

In [111]:
indexes = []

for file in files:
    file_name = file.split('.')[0]
    indexes.append(int(file_name))

In [113]:
for index in tqdm(index_to_delete):
    del dictionary[int(index)]

In [114]:
len(dictionary)

92640

In [115]:
for key in tqdm(list(dictionary)):
    if key not in indexes:
        del dictionary[key] 

In [116]:
len(dictionary)

42688

In [117]:
df_files_static_analysis = pd.DataFrame.from_dict(dictionary, orient='index')

In [119]:
df_files_static_analysis = df_files_static_analysis.drop_duplicates().reset_index(drop=True)
df_files_static_analysis

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,46270581,struct guest {\n char last_name[30];\n...
2,33786645,#include <string.h>\n#include <conio.h.>\n...
3,25411646,void callback (struct Request req)\n{\n cha...
4,44140242,#include <stdio.h>\n#include <stdlib.h>\n#incl...
...,...,...
42683,64567761,"int main(){\n int n;\n int price,q;\n ..."
42684,30573587,"struct tree* buildTree2(char in[], char post[]..."
42685,61468380,#include <stdio.h>\n#include <string.h>\n\nint...
42686,51092098,#include<stdio.h>\n#include<stdlib.h> /* The l...


In [121]:
df_files_static_analysis.to_csv('../Datasets Code Snippets/code_snippets_static_analysis.csv', index=False)

# Detecting vulnerabilities

In [4]:
df_files_static_analysis = pd.read_csv('../Datasets Code Snippets/code_snippets_static_analysis.csv')
df_files_static_analysis

,id,code
0,30877241,void list_file(char* directory) {\n DIR *d;\n...
1,46270581,struct guest {\n char last_name[30];\n...
2,33786645,#include <string.h>\n#include <conio.h.>\n...
3,25411646,void callback (struct Request req)\n{\n cha...
4,44140242,#include <stdio.h>\n#include <stdlib.h>\n#incl...
...,...,...
42683,64567761,"int main(){\n int n;\n int price,q;\n ..."
42684,30573587,"struct tree* buildTree2(char in[], char post[]..."
42685,61468380,#include <stdio.h>\n#include <string.h>\n\nint...
42686,51092098,#include<stdio.h>\n#include<stdlib.h> /* The l...


In [134]:
for index in tqdm(range(df_files_static_analysis.shape[0])):
    file_name = f"{index}.cpp"
    with open("code_static_analysis/"+file_name, "w", encoding="utf-8") as f:
        f.write(df_files_static_analysis.iloc[index, 1])

In [21]:
result_static_analysis = dict()
index = 0

for file in tqdm(os.listdir('code_static_analysis')):
    try:
        i = int(file.split('.')[0])
        proc = subprocess.run(["cppcheck", "--enable=style", "--template={cwe}:{file}:{line}:{severity}:{code}:{message}", "code_static_analysis/"+file], capture_output=True)
        result = proc.stderr.decode('utf-8')
        if result:
            items = result.split('\r\n')
            items = list(filter(None, items))

            for x, y in zip(*[iter(items)] * 2):
                row = x + y
                row = row.split(':')
                result_static_analysis[index]={'id_post': df_files_static_analysis.iloc[i, :]['id'], 'cwe':row[0], 'file': file, 'line': row[2], 'severity': row[3],
                                      'code': row[4], 'message': row[5]}
                index = index + 1
        else:
            result_static_analysis[index]={'id_post': df_files_static_analysis.iloc[i, :]['id'], 'cwe':0, 'file': file, 'line': '', 'severity': '',
                                      'code': '', 'message': ''}
            index = index + 1
    except UnicodeDecodeError:
        print(file)
        break;

In [22]:
len(result_static_analysis) #todo remove 'syntax error' in message

88680

In [64]:
df_result_static_analysis = pd.DataFrame.from_dict(result_static_analysis, orient='index')
df_result_static_analysis

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,46270581,0,1.cpp,,,,
2,29167852,0,10.cpp,,,,
3,21094340,0,100.cpp,,,,
4,35543886,0,1000.cpp,,,,
...,...,...,...,...,...,...,...
88675,46064829,398,9997.cpp,15,style,check = ((test *) check); ^,C-style pointer casting
88676,46064829,398,9997.cpp,16,style,check2 = (test *)check; ^,C-style pointer casting
88677,46064829,398,9997.cpp,21,style,((test *)check)->a = 2; //this also works ^,C-style pointer casting
88678,54440110,398,9998.cpp,10,style,int *array = (int *)malloc(x * sizeof(int)); ...,C-style pointer casting


In [79]:
print("Posts without vulnerabilities: " )
df_result_static_analysis.loc[df_result_static_analysis['severity']=='']

Posts without vulnerabilities: 


,id_post,cwe,file,line,severity,code,message
1,46270581,0,1.cpp,,,,
2,29167852,0,10.cpp,,,,
3,21094340,0,100.cpp,,,,
4,35543886,0,1000.cpp,,,,
5,50084730,0,10000.cpp,,,,
...,...,...,...,...,...,...,...
88669,39382760,0,9991.cpp,,,,
88672,52920852,0,9994.cpp,,,,
88673,52920852,0,9995.cpp,,,,
88674,37165553,0,9996.cpp,,,,


In [85]:
severities = pd.unique(df_result_static_analysis['severity'])
severities = list(filter(None, severities))

for severity in severities:
    to_delete = df_result_static_analysis.loc[df_result_static_analysis['cwe']=='0'].loc[df_result_static_analysis['severity']==severity].index
    df_result_static_analysis = df_result_static_analysis.drop(to_delete)
df_result_static_analysis

,id_post,cwe,file,line,severity,code,message
0,30877241,398,0.cpp,3,style,struct dirent *dir; ^,The scope of the variable 'dir' can be reduced.
1,46270581,0,1.cpp,,,,
2,29167852,0,10.cpp,,,,
3,21094340,0,100.cpp,,,,
4,35543886,0,1000.cpp,,,,
...,...,...,...,...,...,...,...
88675,46064829,398,9997.cpp,15,style,check = ((test *) check); ^,C-style pointer casting
88676,46064829,398,9997.cpp,16,style,check2 = (test *)check; ^,C-style pointer casting
88677,46064829,398,9997.cpp,21,style,((test *)check)->a = 2; //this also works ^,C-style pointer casting
88678,54440110,398,9998.cpp,10,style,int *array = (int *)malloc(x * sizeof(int)); ...,C-style pointer casting


In [96]:
print("Count by severity:")

for severity in severities:
    print(severity+": ",df_result_static_analysis.loc[df_result_static_analysis['severity']==severity].count()[0])
print("Without vulnerabilities: ",df_result_static_analysis.loc[df_result_static_analysis['severity']==''].count()[0])

Count by severity:
style:  39966
error:  10395
portability:  2379
performance:  44
Without vulnerabilities:  17398


In [97]:
df_result_static_analysis.to_csv('Results/results_static_analysis.csv', index=False)